In [1]:
import sys
import socket
from struct import *
import zlib
import time
import pandas as pd
import numpy as np

In [2]:
def flatten_tuple(nested_tuple):
    flat_list = []
    for item in nested_tuple:
        if isinstance(item, (list, tuple)):
            flat_list.extend(flatten_tuple(item))
        elif isinstance(item, np.ndarray):
            flat_list.extend(item.flatten().tolist())
        else:
            flat_list.append(item)
    return flat_list

class ReqMsg:
    def __init__(self, request_id=0):
        self.msg_id = 0
        self.request_id = request_id
        msg_time = time.time()
        self.time = int(msg_time)
        self.time_ns = int((msg_time % 1) * 1e9)
        self.data_size_bytes = 0
        self.data_info = (0, ) * 10

    def pack(self, *args):
        b_msg = pack("=2B2iI10B", *flatten_tuple(tuple(vars(self).values())))
        if self.request_id == 1:
            b_msg = b_msg + pack('996x') #dummy
        elif self.request_id == 2: #velocity
            b_msg = b_msg + pack('=2iBf2B981x', *flatten_tuple(args))
        elif self.request_id == 3: #position
            b_msg = b_msg + pack('=2iBf2B981x', *flatten_tuple(args))
        elif self.request_id == 4: #velocity vectro
            b_msg = b_msg + pack('=iB200f191x', *flatten_tuple(args))
        elif self.request_id == 5: #position vectro
            b_msg = b_msg + pack('=iB200f191x', *flatten_tuple(args))
        elif self.request_id == 255:
            b_msg = b_msg + pack('996')
        b_msg = b_msg + pack('I', int(hex(zlib.crc32(b_msg) & 0xffffffff), base=16))
        self.msg_id = self.msg_id + 1
        return b_msg

In [3]:
socket_address = ('127.0.0.1', 1551)

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, 0)
msg = ReqMsg(1)

In [4]:
def vec_gen(start, end, number):
    vec = [0] * 200
    if number > 200:
        number = 200
    profile = np.linspace(start, end, number)
    for i in range(number):
        vec[i] = profile[i]
    return vec

In [19]:
msg.request_id = 4
num = 30
vec = vec_gen(10, 5, num)

s.sendto(msg.pack(-1, num, vec), socket_address)

1024

In [64]:
s.close()